# Workflow to Validate NISAR L2 Transient Displacement Requirements

**Original code authored by:** NISAR Science Team Members and Affiliates  

*May 13, 2022*

*NISAR Solid Earth Team*

<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. And all subsequent sections NEED to be run in order.
</div>

<hr/>

<hr/>

## Table of Contents: <a id='TOC'></a>

[**Environment Setup**](#setup)
- [Load Python Packages](#load_packages)
- [Define CalVal Site and Parameters](#set_calval_params)
- [Set Directories and Files](#set_directories)

[**1. Download and Prepare Interferograms**](#prep_ifg)
[Executed in ARIA_prep]

[**2. Selection of Interferograms**](#transient_select_ifg)
- [2.1.  Validate/Modify Interferogram Network](#transient_crop_ifg)
- [2.2.  Modify Reference Point](#transient_ref_pt)

[**3. Optional Corrections**](#opt_correction)
- [3.1. Solid Earth Tide Correction](#solid_earth)
- [3.2 Ionosphere Correction](#iono_corr)
- [3.3. Tropospheric Delay Correction](#tropo_corr)
- [3.4. Phase Deramping ](#phase_deramp)
- [3.5. Topographic Residual Correction ](#topo_corr)

[**4. Make GNSS LOS Measurements**](#transient_gnss_los)
- [4.3. Find Collocated GNSS Stations](#transient_co_gnss)  
- [4.4. Make GNSS LOS Measurements](#transient_gnss_los2) 
- [4.5. Make GNSS and InSAR Relative Displacements](#transient_gnss_insar)

[**5. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison**](#transient_validation1)
- [5.1. Pair up GNSS stations and make measurement residuals](#transient_pair1)
- [5.2. Validate the requirement based on binned measurement residuals](#transient_bin1)
- [5.3. Result visulazation](#transient_result1)
- [5.3. Conclusion](#transient_conclusion1)

[**6. NISAR Validation Approach 2: Noise Level Validation**](#transient_validation2)
- [6.1. Randomly sample pixels and pair them up](#transient_pair2)
- [6.2. Validate the requirement based on binned measurement residuals](#transient_bin2)
- [6.3. Result visulazation](#transient_result2)
- [6.3. Conclusion](#transient_conclusion2)

[**Appendix: GNSS Position Plots**](#transient_appendix)

<hr/>

<br>
<hr>

<a id='#setup'></a>
## Environment Setup

### Load Python Packages <a id='#load_packages'></a>

In [ ]:
# Load packages
import copy
import glob
import math
import os
import random
import subprocess
from datetime import datetime as dt
from pathlib import Path
import json

import numpy as np
import pandas as pd
import pyproj
from matplotlib import pyplot as plt
from mintpy import smallbaselineApp
from mintpy.objects import gnss
from mintpy.utils import readfile, utils as ut, network
from mintpy.cli import view

from solid_utils.sampling import load_geo, samp_pair
from solid_utils.plotting import display_validation_table, \
    display_coseismic_validation as display_transient_validation
from solid_utils.configs import update_reference_point
from solid_utils.corrections import run_cmd, pairwise_stack_from_timeseries
from solid_utils.saving import save_results

### Define Calval Site and Parameters <a id='set_calval_params'></a>

In [ ]:
# === Basic Configuration ===
site = "test"
requirement = "Transient"
dataset = 'ARIA_S1_new' # For Sentinel-1 testing with aria-tools
aria_gunw_version = "3_0_1"

rundate = "20250826"  # Date of this Cal/Val run
version = "1b"         # Version of this Cal/Val run
custom_sites = "/home/jovyan/my_sites.txt"  # Path to custom site metadata

# === Username Detection / Creation ===
user_file = "/home/jovyan/me.txt"
if os.path.exists(user_file):
    with open(user_file, "r") as f:
        you = f.readline().strip()
else:
    you = input("Please type a username for your Cal/Val outputs: ").strip()
    with open(user_file, "w") as f:
        f.write(you)

# === Load Cal/Val Site Metadata ===
try:
    with open(custom_sites, "r") as f:
        sitedata = json.load(f)
    site_info = sitedata["sites"][site]
except (FileNotFoundError, json.JSONDecodeError) as e:
    raise RuntimeError(f"Failed to load site metadata from {custom_sites}: {e}")
except KeyError:
    raise ValueError(f"Site ID '{site}' not found in {custom_sites}")

print(f"Loaded site: {site}")

# === Plot Parameters ===
vmin, vmax = -50, 50  # mm
cmap = plt.get_cmap('RdBu')

### Set Directories and Files <a id='set_directories'></a>

In [ ]:
# === Define Cal/Val Directory Structure ===
BASE_DIR = "/scratch/nisar-st-calval-solidearth"
site_dir = os.path.join(BASE_DIR, dataset, site)
work_dir = os.path.join(site_dir, requirement, you, rundate, f"v{version}")
gunw_dir = os.path.join(site_dir, "products")
mintpy_dir = os.path.join(work_dir, "MintPy")
weather_dir = os.path.join(site_dir)

# === Home directory for saving reports ===
home_dir = os.path.join("/home/jovyan/validation/", site, requirement, rundate, f"v{version}")
if not os.path.exists(home_dir):
    os.makedirs(home_dir)

# === Log Directory Paths ===
print(f"  Work directory: {work_dir}")
print(f"  GUNW directory: {gunw_dir}")
print(f"MintPy directory: {mintpy_dir}")

# === Check MintPy Directory Existence ===
if not os.path.exists(mintpy_dir):
    print("\nERROR: Stop! MintPy processing directory is missing.")
    print("This may indicate the prep notebook has not been run.")
    print("Missing path:", mintpy_dir, "\n")
else:
    os.chdir(mintpy_dir)

# === Set Expected MintPy Filenames ===
ifgs_file = os.path.join(mintpy_dir, "inputs/ifgramStack.h5")
geom_file = os.path.join(mintpy_dir, "inputs", "geometryGeo.h5")
msk_file  = os.path.join(work_dir, "mask", "esa_world_cover_2021.msk")
config_file = os.path.join(mintpy_dir, site_info.get('calval_location') + '.cfg')

In [ ]:
configs = readfile.read_template(config_file)
print('#' * 10, "MintPy Configs", '#' * 10)
for key, value in configs.items():
    print(f"{key}: {value}")

<br>
<hr>

<a id='#prep_ifg'></a>
## 1. Download and Prepare Interferograms 
Executed in *ARIA_prep* notebook

<br>
<hr>

<a id='#transient_select_ifg'></a>
## 2. Selection of Interferograms

### 2.1. Validate/Modify Interferogram Network <a id='transient_crop_ifg'></a>

**NOTE:** If the interferogram has a resolution lower than 100 m, we need multi-look the interferogram phase values before calculating the empirical semivarigram.

Load the date of interferograms into Python:

In [ ]:
# Formulate ifgramStack file
ifgramStack_file = os.path.join(mintpy_dir, 'inputs/ifgramStack.h5')

# Modify network - base command
command = f"modify_network.py {ifgramStack_file} -t {config_file} "

# Check whether exclusions specified in my_sites file
if site_info.get('ifgExcludePair') not in [None, 'auto', 'no']:
    command += f" --exclude-ifg {site_info.get('ifgExcludePair')}"

if site_info.get('ifgExcludeDate') not in [None, 'auto', 'no']:
    command += f" --exclude-date {site_info.get('ifgExcludeDate')}"

if site_info.get('ifgExcludeIndex') not in [None, 'auto', 'no']:
    command += f" --exclude-ifg-index {site_info.get('ifgExcludeIndex')} "

# Run command
process = subprocess.run(command, shell=True)

Retrieve the valid date pairs.

In [ ]:
# Retrieve available interferogram date pairs
ifgs_date = network.get_date12_list(ifgs_file, dropIfgram=True)

# Report all available date pairs
print(f"Total {len(ifgs_date)} interferograms available")
for pair in ifgs_date:
    print(f"{'-'.join(pair.split('_'))}")

In [ ]:
# Format date strings into Python datetime objects
_ifgs_date = np.empty_like(ifgs_date, dtype=dt)
for i, pair in enumerate(ifgs_date):
    start_date, end_date = pair.split("_")
    start_date = dt.strptime(start_date, "%Y%m%d")
    end_date = dt.strptime(end_date, "%Y%m%d")
    _ifgs_date[i] = [start_date, end_date]

# Update list of interferogram dates
ifgs_date = _ifgs_date

# Remove temporary list to avoid future confusion
del _ifgs_date

Remove interferograms with time interval other than 12 days:

In [ ]:
# Determine which interferograms to exclude based on 12-day criterion
del_row_index = []
for i in range(ifgs_date.shape[0]):
    time_interval = (ifgs_date[i][1]-ifgs_date[i][0]).days
    if time_interval != 12:
        del_row_index.append(i)
while i<ifgs_date.shape[0]-1:
    if ifgs_date[i][1]==ifgs_date[i+1][0]:
        del_row_index.append(i+1)
        i = i+2
    else:
        i = i+1

# Remove non-12-day interferograms
ifgs_date = np.delete(ifgs_date, del_row_index, 0)

# Report 12-day date pairs
print(f"{len(ifgs_date)} interferograms with 12-day baselines")
for pair in ifgs_date:
    print(f"{pair[0].strftime('%Y%m%d')}-{pair[1].strftime('%Y%m%d')}")

Identify independent interferograms (i.e., selected inteferograms do NOT share common dates):

In [ ]:
# Determine which interferograms to exclude based on independence criterion
del_row_index = []
i = 0
while i<ifgs_date.shape[0]-1:
    if ifgs_date[i][1]==ifgs_date[i+1][0]:
        del_row_index.append(i+1)
        i = i+2
    else:
        i = i+1

# Remove non-independent interferograms from list
ifgs_date = np.delete(ifgs_date, del_row_index, 0)

# Report independent date pairs
print(f"{len(ifgs_date)} independent interferograms (no common date shared)")
for pair in ifgs_date:
    print(f"{pair[0].strftime('%Y%m%d')}-{pair[1].strftime('%Y%m%d')}")

Load the coherence data sets of selected interferograms into memory. Hold off loading the phase values until optional correction layers have been applied.

In [ ]:
# Construct dataset-layer names as lists
coherenceName = [f"coherence-{date[0].strftime('%Y%m%d')}_{date[1].strftime('%Y%m%d')}"
                 for date in ifgs_date]

# Read average filtered spatial coherence
insar_coherence, _ = readfile.read(ifgs_file, datasetName=coherenceName)

### 2.2. Generate Quality Control Mask <a id='generate_mask'></a>

Not implemented. Interferogram masking currently relies on the water mask embedded in the `geometryGeo.h5` file.

### 2.3. Reference Interferograms To Common Lat/Lon <a id='common_latlon'></a>

In [ ]:
if site_info.get('reference_lalo') != 'auto':
    new_lat = site_info.get('reference_lalo').split(',')[0]
    new_lon = site_info.get('reference_lalo').split(',')[1]
    update_reference_point(config_file, new_lat, new_lon) # updates the reference point in MintPy config file
    
# Now reference interferograms to common lat/lon
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
process = subprocess.run(command, shell=True)
os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

In [ ]:
# Visualize the original interferograms
if site_info.get('do_iono') != "False":
    view.main([ifgs_file, '-c', 'RdBu_r', '-m', msk_file])
else: 
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

<a id='transient_opt_correction'></a>
## 3. Optional Interferogram Correction

Phase distortions related to solid earth and ocean tidal effects as well as those due to temporal variations in the vertical stratification of the atmosphere can be mitigated using the approaches described below. At this point, it is expected that these corrections will not be needed to validate the mission requirements, but they may be used to produce the highest quality data products. Typically, these are applied to the estimated time series product rather than to the individual interferograms, since they are a function of the time of each radar acquisition.

### 3.1. Solid Earth Tides Correction <a id='solid_earth'></a>

Not implemented.

In [ ]:
# Automatically set do_SET to False
site_info['do_SET'] = "False"

### 3.2 Ionosphere Correction <a id='iono_corr'></a>

Determine whether the ionosphere correction will be applied.

In [ ]:
if 'do_iono' in site_info.keys() and site_info.get('do_iono') != "False":
    # Input ionosphere stack file  
    iono_stack_file = f"{mintpy_dir}/inputs/ionStack.h5"

    # Check if iono correction file exists
    if os.path.exists(iono_stack_file):
        # Specify file paths
        dirpath, filename = os.path.split(ifgs_file)
        name, ext = os.path.splitext(filename)
        output_ifgs = os.path.join(dirpath, f"{name}_iono{ext}")
    else:
        site_info['do_iono'] = "False"
else:
    site_info['do_iono'] = "False"
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

The `ionStack.h5` dataset is written using the convention ref_repeat and is stored in units of radians. Therefore, the ionosphere correction layers can be directly subtracted from the ifgramStack.h5 unwrapped phase values to apply the ionosphere correction.

In [ ]:
# Set reference point of ionosphere file
if site_info.get('do_iono')!= "False":
    # Run difference
    run_cmd(f"reference_point.py {iono_stack_file} -r {ifgs_file}",
            desc="Apply iono correction to IFG stack")
    

else:
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

In [ ]:
# Apply correction if flag is set to True
if site_info.get('do_iono')!= "False":
    # Run difference
    run_cmd(f"diff.py {ifgs_file} {iono_stack_file} -o {output_ifgs}",
            desc="Apply iono correction to IFG stack")
    
    # Update filename
    ifgs_file = output_ifgs
else:
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

In [ ]:
# Visualize the corrections
if site_info.get('do_iono') != "False":
    view.main([iono_stack_file, '-c', 'RdBu_r', '-m', msk_file])
else: 
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

In [ ]:
# Visualize the corrected interferograms
if site_info.get('do_iono') != "False":
    view.main([ifgs_file, '-c', 'RdBu_r', '-m', msk_file])
else: 
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

### 3.3. Tropospheric Delay Correction <a id='tropo_corr'></a>

Optional atmospheric correction utilizes the PyAPS (Jolivet et al., 2011, Jolivet and Agram, 2012) module within GIAnT (or eventually a merged replacement for GIAnT and MintPy). PyAPS is well documented, maintained and can be freely downloaded. PyAPS is included in GIAnT distribution). PyAPS currently includes support for ECMWF’s ERA-Interim, NOAA’s NARR and NASA’s MERRA weather models. A final selection of atmospheric models to be used for operational NISAR processing will be done during Phase C.

Tropospheric delay maps are produced from atmospheric data provided by Global Atmospheric Models. This method aims to correct differential atmospheric delay correlated with the topography in interferometric phase measurements. Global Atmospheric Models (hereafter GAMs)... provide estimates of the air temperature, the atmospheric pressure and the humidity as a function of elevation on a coarse resolution latitude/longitude grid. In PyAPS, we use this 3D distribution of atmospheric variables to determine the atmospheric phase delay on each pixel of each interferogram.

The absolute atmospheric delay is computed at each SAR acquisition date. For a pixel a_i at an elevation z at acquisition date i, the four surrounding grid points are selected and the delays for their respective elevations are computed. The resulting delay at the pixel a_i is then the bilinear interpolation between the delays at the four grid points. Finally, we combine the absolute delay maps of the InSAR partner images to produce the differential delay maps used to correct the interferograms.

In [ ]:
if 'do_tropo' in site_info.keys() and site_info.get("do_tropo") != "False":
    if site_info.get("tropo_model") != "HRRR":
        # ERA5-based correction
        tropo_source = "ERA5"
        tropo_cor_file = os.path.join(mintpy_dir, "inputs", f"{tropo_source}.h5")

    else:
        # HRRR-based correction
        tropo_source = "HRRR_ARIA"
        tropo_cor_file = os.path.join(mintpy_dir, "inputs", f"{tropo_source}.h5")

    print(f"Troposphere Correction dataset: {tropo_cor_file:s}")

    # Check it tropo correction file exists
    if os.path.exists(tropo_cor_file):
        dirpath, filename = os.path.split(ifgs_file)
        name, ext = os.path.splitext(filename)
        output_ifgs = os.path.join(dirpath, f"{name}_{tropo_source}{ext}")
        print('#'*10, 'Troposphere Correction set to True', '#'*10)
    else:
        site_info['do_tropo'] = "False"

else:
    site_info['do_tropo'] = "False"

if site_info['do_tropo'] == "False":
    print("#" * 10, "Troposphere Correction set to False", "#" * 10)

Troposphere correction layer datasets are stored as time-series (one date per acquisition) and in units of meters. To apply the troposphere delay corrections to the `ifgramStack.h5` files, the differential layers must be computed in ref_repeat format, and scaled to units of radians. This is done using the `create_tropo_pairs` function.

In [ ]:
# Create stack of troposphere correction layer pairs following the above sign convention
if site_info.get('do_tropo')!= "False":
    try:
        # Attempt to create a pairwise stack of tropo files
        tropo_stack_file = pairwise_stack_from_timeseries(ifgs_file, tropo_cor_file)
    except:
        print("Tropo stack generation failed. Setting tropo correction to False")

        # Stack creation failed, set tropo correction to false
        site_info['do_tropo'] = "False"

In [ ]:
# Set reference point of troposphere file
if site_info.get('do_tropo')!= "False":
    # Run difference
    run_cmd(f"reference_point.py {tropo_stack_file} -r {ifgs_file}",
            desc="Apply tropo correction to IFG stack")

else:
    print('#'*10, 'Troposphere Correction set to False', '#'*10)

In [ ]:
# Apply correction if flag is set to True
if site_info.get('do_tropo')!= "False":
    # Run difference
    run_cmd(f"diff.py {ifgs_file} {tropo_stack_file} -o {output_ifgs}",
            desc="Apply tropo correction to IFG stack")
    
    # Update filename
    ifgs_file = output_ifgs
else:
    print('#'*10, 'Troposphere Correction set to False', '#'*10)

In [ ]:
# Visualize the corrections
if site_info.get('do_tropo') != "False":
    view.main([tropo_stack_file, '-c', 'RdBu_r', '-m', msk_file])
else: 
    print('#'*10, 'Troposphere Correction set to False', '#'*10)

In [ ]:
# Visualize the corrected interferograms
if site_info.get('do_tropo') != "False":
    view.main([ifgs_file, '-c', 'RdBu_r', '-m', msk_file])
else: 
    print('#'*10, 'Troposphere Correction set to False', '#'*10)

### 3.4. Phase Deramping <a id='phase_deramp'></a>

Not implemented.

### 3.5. Topographic Residual Correction <a id='topo_corr'></a>

Not implemented.

Preliminary summary: we have load all data we need for processing:
- `atr`: metadata, including incident angle, longitude and latitude step width, etc;
- `insar_displacement`: LOS measurement from InSAR;
- `insar_coherence`: coherence value of the interferograms:
- `ifgs_date`: list of date pairs of two SAR images that form a interferogram.

<a id='transient_gnss_los'></a>
# 4. Make GNSS and InSAR Relative Displacements

Read the unwrapped phase values into memory and convert from phase in radians, to displacement in mm. Change default missing phase values in interferograms from 0.0 to `np.nan`.

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook uses the sign convention <b>ref_repeat</b> (e.g., 20190124_20190112). That is, range decrease is positive and therefore "up" is positive. In contrast, MintPy interferograms follow the opposite convention (range increase is positive).
</div>

In [ ]:
# Construct dataset-layer names as lists
unwrapPhaseName = [f"unwrapPhase-{date[0].strftime('%Y%m%d')}_{date[1].strftime('%Y%m%d')}"
                   for date in ifgs_date]

# Read unwrapped phase from selected interferograms
ifgs_unw, insar_metadata = readfile.read(ifgs_file, datasetName=unwrapPhaseName)

# Convert phase to displacement in m and switch convention to positive range decrease
insar_displacement = -ifgs_unw*float(insar_metadata['WAVELENGTH']) / (4*np.pi)

# Convert displacement units from m to mm
insar_displacement = insar_displacement * 1000.

# Read 2D mask array
msk, _ = readfile.read(msk_file, datasetName="waterMask")

# Repeat mask array for each interferogram
msk = np.stack([msk] * insar_displacement.shape[0], axis=0)
    
# Set masked pixels to NaN
insar_displacement[msk == 0] = np.nan
insar_displacement[insar_displacement==0.0] = np.nan

# Clean up phase-only IFGs to avoid future confusion
del ifgs_unw

### 4.1. Not Used

### 4.2. Not Used <a id='empty'></a>

### 4.3. Find Collocated GNSS Stations <a id='transient_co_gnss'></a>

The project will have access to L2 position data for continuous GNSS stations in third-party networks such NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand, located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

Get space and time range for searching GNSS station:

In [ ]:
# Spatial metadata
length, width = int(insar_metadata['LENGTH']), int(insar_metadata['WIDTH'])
lat_step = float(insar_metadata['Y_STEP'])
lon_step = float(insar_metadata['X_STEP'])
N = float(insar_metadata['Y_FIRST'])
W = float(insar_metadata['X_FIRST'])
S = N+lat_step*(length-1)
E = W+lon_step*(width-1)

# Temporal metadata
start_date_gnss = ifgs_date[0,0]
end_date_gnss = ifgs_date[-1,-1]

Search for collocated GNSS stations:

In [ ]:
# GNSS processing source
if 'gnss_source' in sitedata['sites'][site]:
    gnss_source = sitedata['sites'][site]['gnss_source']
else:
    gnss_source = 'UNR'
print(f"GNSS processing source: {gnss_source:s}")

# Start and end dates
start_date = start_date_gnss.strftime('%Y%m%d')
end_date = end_date_gnss.strftime('%Y%m%d')

# Query GNSS sites within geographic and date range
site_names, site_lats, site_lons = gnss.search_gnss(SNWE=(S,N,W,E),
                                                    start_date=start_date,
                                                    end_date=end_date,
                                                    source=gnss_source)
os.chdir(work_dir)
site_names = [str(stn) for stn in site_names]
print(f"Initial list of {len(site_names)} stations used in analysis:")
print(site_names)

### 4.4. Get GNSS Position Time Series <a id='gps_ts'></a>

In this step, the 3D GNSS observations are projected into the satellite LOS. The InSAR observations are averaged over a 3$\times$3 pixel window around the station positions.

<div class="alert alert-block alert-info">
    <b>Note:</b> The number of pixels used in calculating the averaged phase values at the GPS location depends on the resolution of input data.
</div>

Get daily position solutions for GNSS stations:

In [ ]:
# Empty dictionaries to store InSAR and GNSS data
displacement = {}
gnss_time_series = {}
gnss_time_series_std = {}
bad_stn = {}  # stations to toss
pixel_radius = 3  # number of InSAR pixels to average for comparison with GNSS

# Loop through GNSS sites
for counter, site_name in enumerate(site_names):
    gnss_stn = gnss.get_gnss_class(gnss_source)(site = site_name)
    gnss_stn.open(print_msg=False)

    # Download / read the GNSS site displacement and dates
    gnss_stn.read_displacement()
    dates = gnss_stn.dates

    # Count number of dates in time range by looping through interferograms
    for ifg_ndx in range(insar_displacement.shape[0]):
        # Days in interferogram range (should be 12 based on above filtering)
        start_date = ifgs_date[ifg_ndx,0]
        end_date = ifgs_date[ifg_ndx,-1]
        range_days = (end_date - start_date).days

        # Count number of GNSS epochs in IFG date range
        gnss_count = np.histogram(dates, bins=[start_date,end_date])
        gnss_count = int(gnss_count[0])

        # Select GNSS stations based on data completeness
        # Here we hope to select stations with data frequency of 1 day and no interruption
        if range_days == gnss_count - 1:
            # If start_date in dates and end_date in dates, retrieve displacement data
            (disp_gnss_time_series,
             disp_gnss_time_series_std,
             site_latlon) = gnss_stn.get_los_displacement(geom_file,
                                                          start_date=start_date.strftime('%Y%m%d'),
                                                          end_date=end_date.strftime('%Y%m%d'))[1:4]

            # Compute station pixel coordinates
            x_value = round((site_latlon[1] - W)/lon_step)
            y_value = round((site_latlon[0] - N)/lat_step)

            # Displacement from insar observation in the gnss station, averaged
            # Caution: If you expand the radius parameter farther than the bounding grid it will break. 
            disp_insar = insar_displacement[ifg_ndx,
                                            y_value-pixel_radius:y_value+pixel_radius+1, 
                                            x_value-pixel_radius:x_value+pixel_radius+1]

            # Check values in InSAR displacement series are valid
            if np.isfinite(disp_insar).sum() == 0:
                # Ignore station if infinite values
                break

            # Compute mean of window around InSAR pixel, ignoring NaNs
            disp_insar = np.nanmean(disp_insar)

            # Scale GNSS displacement values from m to mm
            disp_gnss_time_series = disp_gnss_time_series*1000
            disp_gnss_time_series_std = disp_gnss_time_series_std*1000

            # Store time-series displacements to dicts, labeled by IFG index and site name
            gnss_time_series[(ifg_ndx, site_name)] = disp_gnss_time_series
            gnss_time_series_std[(ifg_ndx, site_name)] = disp_gnss_time_series_std
            displacement[(ifg_ndx, site_name)] = list(site_latlon)
            disp_gnss = disp_gnss_time_series[-1] - disp_gnss_time_series[0]

            displacement[(ifg_ndx, site_name)].append(disp_gnss)
            displacement[(ifg_ndx, site_name)].append(disp_insar)
        else:
            try:
                bad_stn[ifg_ndx].append(site_name)
            except:
                bad_stn[ifg_ndx] = [site_name]

Do some data structure transformation:

In [ ]:
# Rearrange GNSS TS and disp values by IFG number and site name
gnss_time_series = dict(sorted(gnss_time_series.items()))
gnss_time_series_std = dict(sorted(gnss_time_series_std.items()))
displacement = dict(sorted(displacement.items()))
bad_stn = dict(sorted(bad_stn.items()))

In [ ]:
# Convert GNSS TS dictionaries to pandas dataframes
gnss_time_series = pd.DataFrame.from_dict(gnss_time_series)
gnss_time_series_std = pd.DataFrame.from_dict(gnss_time_series_std)

In [ ]:
# Convert displacement dictionaries to pandas dataframes
displacement = pd.DataFrame.from_dict(displacement, orient='index',
                                      columns=['lat','lon','gnss_disp','insar_disp'])

# Organize by IFG index and site name
displacement.index = pd.MultiIndex.from_tuples(displacement.index, names=['ifg index','station'])

If there are fewer than 3 GNSS stations, don't conduct comparison:

In [ ]:
# Drop IFGs with fewer than three stations
drop_index = []
for i in displacement.index.get_level_values(0).unique():
    if len(displacement.loc[i]) < 3:
        drop_index.append(i)
displacement=displacement.drop(drop_index)

# ifgs_date after drop for approach 1
ifgs_date_ap1=np.delete(ifgs_date,drop_index,axis=0)

All data needed for approach 1:

In [ ]:
displacement

<div class="alert alert-block alert-info">
    <b>Note:</b> A more general critterion is needed for GNSS station selection. Here the stations with uninterrupted data are selected while, in Secular Requirement Validation, stations are selected by data completeness and standard variation.
</div>

### 4.6. Re-reference GNSS and InSAR <a id='reference'></a>

Here we randomly select one reference site and make both the GNSS and InSAR measurements relative to that reference to remove a constant offset.

In [ ]:
# Read reference site
gnss_ref_site_name = sitedata['sites'][site]['gps_ref_site_name']
print(f"Using reference site: {gnss_ref_site_name:s}")

# Loop through interferograms to re-reference
for ifg_ndx in displacement.index.get_level_values(0).unique():
    # Determine reference site
    if gnss_ref_site_name in ['auto', 'random']:
        # Choose random GNSS site
        gnss_ref_site_name = random.choice(displacement.loc[ifg_ndx].index.unique())
        print(f"Ifg {ifg_ndx} reference site: {gnss_ref_site_name}")

    # Remove reference site values from GNSS and InSAR displacements
    displacement.loc[ifg_ndx, 'gnss_disp'] = displacement.loc[ifg_ndx, 'gnss_disp'].values \
            - displacement.loc[(ifg_ndx, gnss_ref_site_name), 'gnss_disp']
    displacement.loc[ifg_ndx, 'insar_disp'] = displacement.loc[ifg_ndx, 'insar_disp'].values \
            - displacement.loc[(ifg_ndx, gnss_ref_site_name), 'insar_disp']

    # Reference point pixel coordinates
    ref_x_value = round((displacement.loc[(ifg_ndx, gnss_ref_site_name),'lon'] - W)/lon_step)
    ref_y_value = round((displacement.loc[(ifg_ndx, gnss_ref_site_name),'lat'] - N)/lat_step)

    # InSAR displacement values at site location for re-referencing
    ref_disp_insar = insar_displacement[ifg_ndx,
                                        ref_y_value-pixel_radius:ref_y_value+1+pixel_radius, 
                                        ref_x_value-pixel_radius:ref_x_value+1+pixel_radius]

    # Re-referenced mean value at site location
    ref_disp_insar = np.nanmean(ref_disp_insar)

    # Subtract reference value from InSAR displacement
    insar_displacement[ifg_ndx] -= ref_disp_insar

Plot GNSS stations on InSAR displacement fields

In [ ]:
# Set color values
cmap_obj = copy.copy(plt.get_cmap(cmap))

vmin = vmin if vmin is not None else np.nanmin(insar_displacement)
vmax = vmax if vmax is not None else np.nanmax(insar_displacement)

# Loop through interferograms
gnss_insar_figs = []
for ifg_ndx in displacement.index.get_level_values(0).unique():
    fig, ax = plt.subplots(figsize = (8,8))
    img1 = ax.imshow(insar_displacement[ifg_ndx],
                     cmap=cmap_obj, vmin=vmin, vmax=vmax, interpolation='nearest',
                     extent=(W, E, S, N))
    ax.set_title(f"{ifgs_date[ifg_ndx,0].strftime('%Y%m%d')}-{ifgs_date[ifg_ndx,1].strftime('%Y%m%d')}")
    cbar1 = fig.colorbar(img1, ax=ax, orientation='horizontal')
    cbar1.set_label('LOS displacement [mm]')

    for site_name in displacement.loc[ifg_ndx].index:
        lon, lat = displacement.loc[(ifg_ndx, site_name), 'lon'], displacement.loc[(ifg_ndx, site_name), 'lat']
        color = cmap((displacement.loc[(ifg_ndx, site_name), 'gnss_disp']-vmin)/(vmax-vmin))
        ax.scatter(lon, lat, s=8**2, color=color, edgecolors='k')
        ax.annotate(site_name, (lon,lat), color='black')

    # Append figure to list
    gnss_insar_figs.append(fig)

<br>
<hr>

<a id='validation1'></a>
## 5. Validation Method 1: GNSS-InSAR Direct Comparison

<a id='transient_pair1'></a>
### 5.1, 5.2. Make GNSS-InSAR Velocity Residuals at GNSS Station Locations

We first pair up all GNSS stations and compare the relative measurement from both GNSS and InSAR. 

In [ ]:
# Empty dictionaries for GNSS and InSAR measurements, etc.
insar_disp = {}
gnss_disp = {}
ddiff_dist = {}
ddiff_disp = {}
abs_ddiff_disp = {}

# Define ellipsoid for distance calculation
geod = pyproj.Geod(ellps="WGS84")

# Loop through interferograms
for ifg_ndx in displacement.index.get_level_values(0).unique():
    displacement_i = displacement.loc[ifg_ndx]
    insar_disp_i = []
    gnss_disp_i = []
    ddiff_dist_i = []
    ddiff_disp_i = []

    # Loop through site pairs
    for sta1 in displacement_i.index:
        for sta2 in displacement_i.index:
            if sta2 == sta1:
                break

            # Compute InSAR and GNSS displacement residuals
            insar_disp_i.append(displacement_i.loc[sta1, 'insar_disp'] \
                                - displacement_i.loc[sta2, 'insar_disp'])
            gnss_disp_i.append(displacement_i.loc[sta1, 'gnss_disp'] \
                               - displacement_i.loc[sta2, 'gnss_disp'])

            # Compute double-difference residual
            ddiff_disp_i.append(gnss_disp_i[-1] - insar_disp_i[-1])

            # Compute distance between sites
            _, _, distance = geod.inv(displacement_i.loc[sta1,'lon'], displacement_i.loc[sta1,'lat'],
                                      displacement_i.loc[sta2,'lon'], displacement_i.loc[sta2,'lat'])
            distance = distance / 1000  # convert unit from m to km

            # Record double difference
            ddiff_dist_i.append(distance)

    # Record all site-to-site values within an IFG
    insar_disp[ifg_ndx] = np.array(insar_disp_i)
    gnss_disp[ifg_ndx] = np.array(gnss_disp_i)
    ddiff_dist[ifg_ndx] = np.array(ddiff_dist_i)
    ddiff_disp[ifg_ndx] = np.array(ddiff_disp_i)
    abs_ddiff_disp[ifg_ndx] = abs(np.array(ddiff_disp_i))

Plot Absolute Displacement Residuals As a Function of Distance:

<a id='transient_bin1'></a>
## 5.3 Validate Requirement Based on Binned Measurement Residuals

In [ ]:
# Set requirement thresholds
transient_distance_rqmt = (0.1, 50)  # distances for evaluation
transient_threshold_rqmt = lambda L: 3 * (1 + np.sqrt(L))  # coseismic threshold in mm

n_bins = 10  # number of distance bins for analysis
threshold = 0.683  # fraction of Gaussian normal distribution for pass/fail

# Loop through interferograms
method1_validation_figs = []
site_loc = sitedata['sites'][site]['calval_location']
for ifg_ndx in displacement.index.get_level_values(0).unique():
    # Start and end dates as strings
    start_date = ifgs_date[ifg_ndx,0].strftime('%Y%m%d')
    end_date = ifgs_date[ifg_ndx,1].strftime('%Y%m%d')

    # Validation figure and assessment
    _, validation_fig_method1 = display_transient_validation(ddiff_dist[ifg_ndx], abs_ddiff_disp[ifg_ndx],
                                                             site_loc, start_date, end_date,
                                                             requirement=transient_threshold_rqmt,
                                                             distance_rqmt=transient_distance_rqmt,
                                                             n_bins=n_bins,
                                                             threshold=threshold,
                                                             sensor='Sentinel-1',
                                                             validation_type=requirement.lower(),
                                                             validation_data='GNSS')
    method1_validation_figs.append(validation_fig_method1)

In [ ]:
# Reformat double differences as list
ddiff_dist_ap1 = list(ddiff_dist.values())
abs_ddiff_disp_ap1 = list(abs_ddiff_disp.values())

We have got all needed data for approach 1:
- `ddiff_dist_ap1`: distance of GNSS pairs,
- `abs_ddiff_disp_ap1`: absolute value of measurement redisuals,
- `ifgs_date_ap1`: list of date pairs of two SAR images that form a interferogram.

In [ ]:
# Define number of interferograms
n_ifgs = len(ddiff_dist_ap1)
print(f"Analyzing {n_ifgs} interferograms")

Bin all measurement residuals to check if they pass the requirements or not:

In [ ]:
# Define bins over distance requirement
n_bins = 10
bins = np.linspace(0.1, 50.0, num=n_bins+1)

In [ ]:
# Pre-allocate array for number of points for each IFG and bins
n_all = np.empty([n_ifgs, n_bins+1], dtype=int)

# Pre-allocate array for number of points that pass based on requirement
n_pass = np.empty([n_ifgs,n_bins+1], dtype=int)

# Loop through interferograms
for i in range(n_ifgs):
    # Determine bin indices
    inds = np.digitize(ddiff_dist_ap1[i], bins)

    # Loop through bins
    for j in range(1,n_bins+1):
        # Evaluate requirement for the i-th IFG and j-th distance bin
        rqmt = 3*(1+np.sqrt(ddiff_dist_ap1[i][inds==j]))

        # Relative measurement of i-th IFG and j-th distance bin
        rem = abs_ddiff_disp_ap1[i][inds==j]
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)

    # Total number of residuals
    n_all[i,-1] = np.sum(n_all[i,0:-2])

    # Number of residuals that pass requirement
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [ ]:
# Ratio of double-difference residuals that pass requirement
ratio = n_pass / n_all

# Define threshold of data points in a bin that must pass
thresthod = 0.683

# The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio > thresthod

<a id='transient_result1'></a>
## Result visualization

Convert the result to pandas DataFrame for better visulization:

In [ ]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

index = []
for i in range(len(ifgs_date_ap1)):
    index.append(ifgs_date_ap1[i,0].strftime('%Y%m%d')+'-'+ifgs_date_ap1[i,1].strftime('%Y%m%d'))

n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns,index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns,index=index)

Number of data points in each bin:

In [ ]:
n_all_pd

Number of data points that below the curve:

In [ ]:
n_pass_pd

Percentage of pass:

In [ ]:
# Stylized pandas table
validation_table_method1 = ratio_pd.style
validation_table_method1.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
validation_table_method1.set_td_classes(success_or_fail_pd)

<a id='transient_conclusion1'></a>
## Conclusion

In [ ]:
percentage = np.count_nonzero(ratio_pd['total'] > thresthod) / n_ifgs

method_summary = f"Percentage of interferograms passes the requirement: {percentage}"

if percentage >= 0.70:
    method_summary += "\nThe interferogram stack passes the requirement."
else:
    method_summary += "\nThe interferogram stack fails the requirement."

print(method_summary)

In [ ]:
# Save Method 1 results to file
run_date = dt.now().strftime('%Y%m%dT%H%M%S')
save_fldr = f"{run_date}-Transient-Method1"
save_dir = os.path.join(mintpy_dir, save_fldr)

save_params = {
    'save_dir': save_dir,
    'run_date': run_date,
    'requirement': requirement,
    'site': site,
    'method': '1',
    'sitedata': sitedata['sites'][site],
    'gnss_insar_figs': gnss_insar_figs,
    'validation_figs': method1_validation_figs,
    'validation_table': validation_table_method1,
    'summary': method_summary
}
save_results(**save_params)

# Save the report in the home directory as well
save_params['save_dir'] = home_dir
save_results(**save_params)

<div class="alert alert-warning">
Approach 1 final result for CentralValleyA144: around 79% of interferograms passes the requirement.
</div>

<br>
<hr>

<a id='transient_validation2'></a>
## 6. Validation Approach 2: Noise Level Validation

In this validation (Approach #2), we evaluate the estimated secular deformation rate (Requirements 658) or co-seismic displacement (Requirement 660) from time series processing or the individual unwrapped interferogram (Requirement 663) over selected cal/val areas with negligible deformation. Any estimated deformation should thus be treated as noise and our goal is to evaluate the significance of this noise. In general, noise in the modeled displacement or the unwrapped interferogram is anisotropic, but here we neglect this anisotropy. Also, we assume the noise is stationary.

We first randomly sample measurements and pair up sampled pixel measurements. For each pixel-pair, the difference of their measurement becomes:
$$d\left(r\right)=|(f\left(x\right)-f\left(x-r\right))|$$
Estimates of $d(r)$ from all pairs are binned according to the distance r. In each bin, $d(r)$ is assumed to be a normal distribution.

**Note:** Now we simply assume there is no deformation in this study area and time interval. But in fact, it is hard to find a enough large area without any deformation. An more realistic solution is to apply a mask to mask out deformed regions.

### 6.1. Read InSAR Array and Mask Pixels with no Data <a id='array_mask'></a>

In [ ]:
# Construct dataset-layer names as lists
unwrapPhaseName = [f"unwrapPhase-{date[0].strftime('%Y%m%d')}_{date[1].strftime('%Y%m%d')}"
                   for date in ifgs_date]

# Read unwrapped phase from selected interferograms
ifgs_unw, insar_metadata = readfile.read(ifgs_file, datasetName=unwrapPhaseName)

# Convert phase to displacement in m and switch convention to positive range decrease
insar_displacement = -ifgs_unw*float(insar_metadata['WAVELENGTH']) / (4*np.pi)

# Convert displacement units from m to mm
insar_displacement = insar_displacement * 1000.

# Read 2D mask array
msk, _ = readfile.read(msk_file, datasetName="waterMask")

# Repeat mask array for each interferogram
msk = np.stack([msk] * insar_displacement.shape[0], axis=0)
    
# Set masked pixels to NaN
insar_displacement[msk == 0] = np.nan
insar_displacement[insar_displacement==0.0] = np.nan

# Clean up phase-only IFGs to avoid future confusion
del ifgs_unw

# Define number of interferograms
n_ifgs = len(ddiff_dist_ap1)
print(f"Analyzing {n_ifgs} interferograms")

Mask Pixels with Low Coherence (optional)

In [ ]:
#insar_displacement[insar_coherence <0.6] = np.nan

Plot the coherence and InSAR measurements:

In [ ]:
cmap_obj = copy.copy(plt.get_cmap('gray'))

for ifg_ndx in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.imshow(insar_coherence[ifg_ndx], cmap=cmap_obj, interpolation='nearest', extent=(W, E, S, N))
    ax.set_title(f"Coherence "
                 f"\n Date range {ifgs_date[ifg_ndx,0].strftime('%Y%m%d')}-{ifgs_date[ifg_ndx,1].strftime('%Y%m%d')}")
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('coherence')

In [ ]:
cmap_obj = copy.copy(plt.get_cmap(cmap))

for ifg_ndx in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.imshow(insar_displacement[ifg_ndx], cmap=cmap_obj, interpolation='nearest', extent=(W, E, S, N))
    ax.set_title(f"Interferogram "
                 f"\n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('LOS displacement [mm]')

<a id='transient_pair2'></a>
### 6.2. Randomly Sample Pixels and Pair Them Up

Calculate the coordinate for every pixel:

In [ ]:
X0,Y0 = load_geo(insar_metadata)
X0_2d, Y0_2d = np.meshgrid(X0, Y0)

For each interferogram, randomly selected pixels need to be paired up. In order to keep measurements independent, different pixel pairs can not share same pixel. This is achieved by pairing up in sequence, i.e., pairing up pixel number 1 and number 2, 3 and 4...

In [ ]:
# Determine the distance and phase difference between site pairs
dist = []; rel_measure = []
for ifg_ndx in range(n_ifgs):
    dist_i, rel_measure_i = samp_pair(X0_2d, Y0_2d, insar_displacement[ifg_ndx], num_samples=1000000)
    dist.append(dist_i)
    rel_measure.append(rel_measure_i)

Show the statistical property of selected pixel pairs:

In [ ]:
# Plot histogram of distances between pixel pairs
for ifg_ndx in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.hist(dist[ifg_ndx], bins=100)
    ax.set_title(f"Histogram of distance "
                 f"\n Date range {ifgs_date[ifg_ndx,0].strftime('%Y%m%d')}-{ifgs_date[ifg_ndx,1].strftime('%Y%m%d')}")
    ax.set_xlabel(r'Distance ($km$)')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0, 50)

In [ ]:
# Plot histogram of relative measurements
for ifg_ndx in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.hist(rel_measure[ifg_ndx], bins=100)
    ax.set_title(f"Histogram of Relative Measurement "
                 f"\n Date range {ifgs_date[ifg_ndx,0].strftime('%Y%m%d')}-{ifgs_date[ifg_ndx,1].strftime('%Y%m%d')}")
    ax.set_xlabel(r'Relative Measurement ($mm$)')
    ax.set_ylabel('Frequency')

We have got data used of approach 2:
- `dist`: distance of pixel pairs,
- `rel_measure`: relative measurement of pixel pairs,
- `ifgs_date`: list of date pairs of two SAR images that form a interferogram.

<a id='transient_bin2'></a>
### 6.3. Validate the requirement based on binned measurement residuals

In [ ]:
# Define number of interferograms
n_ifgs = len(ddiff_dist_ap1)
print(f"Analyzing {n_ifgs} interferograms")

Bin all measurement residuals to check if they pass the requirements or not:

In [ ]:
# Define bins over distance requirement
n_bins = 10
bins = np.linspace(0.1, 50.0, num=n_bins+1)

In [ ]:
# Number of points for each ifgs and bins
n_all = np.empty([n_ifgs, n_bins+1], dtype=int)

# Number of points pass
n_pass = np.empty([n_ifgs,n_bins+1], dtype=int)

# Loop through interferograms
for i in range(n_ifgs):
    # Determine bin indices
    inds = np.digitize(dist[i], bins)

    # Loop through bins
    for j in range(1, n_bins+1):
        # Evaluate requirement for the i-th IFG and j-th distance bin
        rqmt = 3*(1+np.sqrt(dist[i][inds==j]))  # mission requirement for i-th ifgs and j-th bins

        # Relative measurement of i-th IFG and j-th distance bin
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)

    # Total number of residuals
    n_all[i,-1] = np.sum(n_all[i,0:-2])

    # Number of residuals that pass requirement
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [ ]:
# Ratio of sample pairs that pass requirement
ratio = n_pass / n_all
mean_ratio = np.array([np.mean(ratio[:,:-1],axis=1)])
ratio = np.hstack((ratio,mean_ratio.T))

# Define threshold of data points in a bin that must pass
thresthod = 0.683

#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio > thresthod

In [ ]:
# Set requirement thresholds
transient_distance_rqmt = (0.1, 50)  # distances for evaluation
transient_threshold_rqmt = lambda L: 3 * (1 + np.sqrt(L))  # coseismic threshold in mm

n_bins = 10  # number of distance bins for analysis
threshold = 0.683  # fraction of Gaussian normal distribution for pass/fail


# Loop through interferograms
method2_validation_figs = []
for ifg_ndx in displacement.index.get_level_values(0).unique():
    # Start and end dates as strings
    start_date = ifgs_date[ifg_ndx,0].strftime('%Y%m%d')
    end_date = ifgs_date[ifg_ndx,1].strftime('%Y%m%d')

    # Validation figure and assessment
    _, validation_fig_method2 = display_transient_validation(dist[ifg_ndx], rel_measure[ifg_ndx],
                                 site, start_date, end_date,
                                 requirement=transient_threshold_rqmt,
                                 distance_rqmt=transient_distance_rqmt,
                                 n_bins=n_bins,
                                 threshold=threshold,
                                 sensor='Sentinel-1',
                                 validation_type=requirement.lower(),
                                 validation_data='GNSS')

    method2_validation_figs.append(validation_fig_method2)

<a id='transient_result2'></a>
## Result visualization

Convert the result to pandas DataFrame for better visulization:

In [ ]:
# Format evaluation success/failure as string
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

columns = []
for i in range(n_bins):
    columns.append(f"{bins[i]:.2f}-{bins[i+1]:.2f}")
columns.append('total')

index = []
for i in range(len(ifgs_date)):
    index.append(f"{ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")

n_all_pd = pd.DataFrame(n_all, columns=columns, index=index)
n_pass_pd = pd.DataFrame(n_pass, columns=columns, index=index)
ratio_pd = pd.DataFrame(ratio, columns=columns+['mean'], index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str, columns=columns+['mean'], index=index)

Number of data points in each bin:

In [ ]:
n_all_pd

Number of data points that below the curve:

In [ ]:
n_pass_pd

Ratio of pass:

In [ ]:
# Stylized pandas table
validation_table_method2 = ratio_pd.style
validation_table_method2.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
validation_table_method2.set_td_classes(success_or_fail_pd)

<a id='transient_conclusion2'></a>
## Conclusion

Compared with percentage of total passed pairs, the mean value of percentage of passed pairs in all bin is a better indicator since it gives all bins same weight. 

In [ ]:
percentage = np.count_nonzero(ratio_pd['mean'] > thresthod) / n_ifgs

In [ ]:
print(f"Percentage of interferograms passes the requirement (70%): {percentage}.")
if percentage >= 0.70:
    print('The interferogram stack passes the requirement.')
else:
    print('The interferogram stack fails the requirement.')

In [ ]:
# Save Method 2 results to file
save_fldr = f"{dt.now().strftime('%Y%m%dT%H%M%S')}-Transient-Method2"
save_dir = os.path.join(mintpy_dir, save_fldr)

save_params = {
    'save_dir': save_dir,
    'run_date': run_date,
    'requirement': requirement,
    'site': site,
    'method': '2',
    'sitedata': sitedata['sites'][site],
    'gnss_insar_figs': gnss_insar_figs,
    'validation_figs': method2_validation_figs,
    'validation_table': validation_table_method2,
    'summary': method_summary
}
save_results(**save_params)

# Save the report in the home directory as well
save_params['save_dir'] = home_dir
save_results(**save_params)

<div class="alert alert-warning">
Approach 2 final result for CentralValleyA144: 100% of interferograms passes the requirement.
</div>

<a id='transient_appendix'></a>
# Appendix: InSAR and GNSS Position Plots

The relative position in LOS direction for all GNSS stations are plotted here.

In [ ]:
# Loop through interferograms
for ifg_ndx in displacement.index.get_level_values(0).unique():
    # Initialize figure
    plt.figure(figsize=(11,7))

    # Define range of displacement values
    disp_range = (min([*insar_disp[ifg_ndx],*gnss_disp[ifg_ndx]]), max([*insar_disp[ifg_ndx],*gnss_disp[ifg_ndx]]))

    # Plot histograms of InSAR and GNSS displacement values
    plt.hist(insar_disp[ifg_ndx], bins=100, range=disp_range, color = "green", label='D_InSAR')
    plt.hist(gnss_disp[ifg_ndx], bins=100, range=disp_range, color="orange", label='D_GNSS', alpha=0.5)

    # Format figure
    plt.legend(loc='upper right')
    plt.title(f"Displacements "
              f"\n Date range {ifgs_date[ifg_ndx,0].strftime('%Y%m%d')}-{ifgs_date[ifg_ndx,1].strftime('%Y%m%d')} "
              f"\n Number of station pairs used: {len(insar_disp[ifg_ndx])}")
    plt.xlabel('LOS Displacement (mm)')
    plt.ylabel('Number of Station Pairs')
    plt.show()

In [ ]:
# Loop through interferograms
for ifg_ndx in displacement.index.get_level_values(0).unique():
    # Initialize figure
    plt.figure(figsize=(11,7))

    # Plot histogram
    plt.hist(ddiff_disp[ifg_ndx], bins=100, color='darkblue', linewidth=1, label='D_gnss - D_InSAR')

    # Format figure
    plt.legend(loc='upper right')
    plt.title(f"Residuals"
              f"\n Date range {ifgs_date[ifg_ndx,0].strftime('%Y%m%d')}-{ifgs_date[ifg_ndx,1].strftime('%Y%m%d')}"
              f"\n Number of stations pairs used: {len(ddiff_disp[i])}")
    plt.xlabel('Displacement Residual (mm)')
    plt.ylabel('N Stations')
    plt.show()

In [ ]:
# Loop through interferograms
for ifg_ndx in displacement.index.get_level_values(0).unique():
    # Initialize figure
    plt.figure(figsize=(11,7))

    # Draw distance threshold
    dist_th = np.linspace(min(ddiff_dist[ifg_ndx]), max(ddiff_dist[ifg_ndx]),100)
    acpt_error = 3*(1+np.sqrt(dist_th))

    # Plot residuals
    plt.scatter(ddiff_dist[ifg_ndx], abs_ddiff_disp[ifg_ndx], s=1)
    plt.plot(dist_th, acpt_error, 'r')

    # Format plot
    plt.xlabel("Distance (km)")
    plt.ylabel("Amplitude of Displacement Residuals (mm)")
    plt.title(f"Residuals "
              f"\n Date range {ifgs_date[ifg_ndx,0].strftime('%Y%m%d')}-{ifgs_date[ifg_ndx,1].strftime('%Y%m%d')} "
              f"\n Number of stations pairs used: {len(ddiff_dist[ifg_ndx])}")
    plt.legend(["Measurement", "Mission Reqiurement"])
    plt.show()

In [ ]:
gnss_time_series[1]

In [ ]:
# Loop through interferograms
for ifg_ndx in range(insar_displacement.shape[0]):
    # Define start and end dates
    start_time_str = ifgs_date[ifg_ndx,0].strftime('%Y%m%d')
    end_time_str = ifgs_date[ifg_ndx,1].strftime('%Y%m%d')

    # Loop through stations in IFG
    for site_name in gnss_time_series[ifg_ndx].columns:
        print(f"Plotting GPS postion from {start_time_str} to {end_time_str} at station: {site_name}")

        # Retrieve GNSS time-series
        series = gnss_time_series[ifg_ndx, site_name]

        # Initialize figure
        plt.figure(figsize=(15,5))

        # Plot time-series data
        plt.scatter(pd.date_range(start=ifgs_date[ifg_ndx,0], end=ifgs_date[ifg_ndx,1]),series)

        # Foramt figure
        plt.title(f"station name: {site_name}")
        plt.xlabel('Time')
        plt.ylabel('Relative position in LOS direction (mm)')

        # Save figure
        plt.savefig(os.path.join(work_dir, f"{start_time_str}_{end_time_str}_{site_name}.jpg"))
        plt.close()